In [21]:
##記事が古すぎた！！！
# http://curlyst.hatenablog.com/entry/2016/07/07/001722

import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn import datasets

import chainer
from chainer import Variable, optimizers
import chainer.functions as F
import chainer.links as L

# データインポート
iris = datasets.load_iris()
# irisのデータ見る
pd.DataFrame(iris.data, columns=iris.feature_names)
iris.target
iris.target_names
##

#目的変数
target_data = iris.target.astype(np.int32)

# 説明変数
# https://fisproject.jp/2016/06/data-standardization-using-python/
# 標準化と正規化
tmp_predictor_data = sp.stats.zscore(iris.data, axis=0)
predictor_data =tmp_predictor_data.astype(np.float32)

# 学習データとテストデータに分割
test_size = 30
data_num = len(target_data)

# stratify: クラスを表す行列を設定
# Stratified Sampling
predictor_train, predictor_test, target_train, target_test = train_test_split(predictor_data, target_data, test_size=test_size, stratify=target_data)
train_data_num = len(target_train)
test_data_num = len(target_test)

In [46]:
# モデルの定義
class MultiLayerPerceptron(chainer.Chain):
    '''
        モデルの定義
    '''
    def __init__(self, input_dim, n_units, output_dim, train=True, drop_out_ratio=0.3):
        '''
        コンストラクタ
        '''
        super(MultiLayerPerceptron, self).__init__(
            l1=L.Linear(input_dim, n_units),
            l2=L.Linear(n_units, n_units),
            l3=L.Linear(n_units, output_dim)
        )
        self__train = train
        self.__drop_out = True
        self.drop_out_ratio = drop_out_ratio
    
    def __call__(self, x):
        '''
        順伝播計算
        '''
        drop_out = self.__train and self.__drop_out
        h1 = F.dropout(F.relu(self.l1(x)), train=drop_out, ratio=self.drop_out_ratio)
        h2 = F.dropout(F.relu(self.l2(h1)), train=drop_out, ratio=self.drop_out_ratio)
        y = self.l3(h2)
        
        return y
    
    def __get_train(self):
        return self.__train
    
    def __set_train(self, train):
        self.__train = train
        
    train = property(__get_train, __set_train)
    
    # Dropoutを使用する場合: True
    def __get_drop_out(self):
        return self.__drop_out
    
    def __set_drop_out(self, drop_out):
        '''
        drop outフラグの設定
        '''
        self.__drop_out = drop_out
        
    drop_out = property(__get_drop_out, __set_drop_out)
    

In [47]:
class MiniBatchLearner:
    '''
    ミニバッチによる学習を行うクラス
    '''
    def __init__(self, optimizer, epoch_num, batch_size):
        '''
        コンストラクタ
        '''
        self.__optimizer = None
        self.__epoch_num = None
        self.__batch_size = None
        self.__train_loss = None
        self.__train_acc = None
        self.__test_loss = None
        self.__test_acc = None
        # パラメータの初期化
        self.set_params(optimizer, epoch_num, batch_size)
        self.__init_loss_acc()
        
    def set_params(self, optimizer, epoch_num, batch_size):
        self.__optimizer = optimizer
        self.__epoch_num = epoch_num
        self.__batch_size = batch_size
        
    def __init_loss_acc(self):
        self.__train_loss = []
        self.__train_acc = []
        self.__test_loss = []
        self.__test_acc = []
        
    def learn(self, model, predictor_train_data, target_train_data, drop_out=True):
        '''
        学習の実施
        '''
        self.__init_loss_acc()
        # 学習データのインデックス(ランダム)
        train_data_num = len(target_train_data)
        perm = np.random.permutation(train_data_num)
        
        sum_accuracy = 0
        sum_loss = 0
        
        # 学習モード
        model.predictor.train = True
        model.predictor.drop_out = drop_out
        
        for idx in range(0, train_data_num, self.__batch_size):
            predictor_batch = chainer.Variable(predictor_train_data[perm[idx:idx+self.__batch_size]])
            target_batch = chainer.Variable(target_train_data[perm[idx:idx+self.__batch_size]])
            
            # 勾配を初期化
            model.zerograds()
            # 順伝播させて誤差と精度を算出
            loss = model(predictor_batch, target_batch)
            acc = model.accuracy
            loss.backward()
            self.__optimizer.update()
            
            self.__train_loss.append(loss.data)
            self.__train_acc.append(acc.data)
            sum_loss += float(loss.data) * len(target_batch)
            sum_accuracy += float(acc.data) * len(target_batch)
            
        train_mean_loss = sum_loss / train_data_num
        train_mean_acc = sum_accuracy / train_data_num
        return train_mean_loss, train_mean_acc

    def evaluate(self, model, predictor_test_data, target_test_data):
        sum_accuracy = 0
        sum_loss = 0
        
        model.predictor.train = False
        
        predictor_batch = chainer.Variable(predictor_test_data)
        target_batch = chainer.Variable(target_test_data)
        
        # 順伝播させて誤差と精度を算出
        loss = model(predictor_batch, target_batch)
        acc = model.accuracy
        
        test_data_num = len(target_test_data)
        sum_loss = foat(loss.data) * test_data_num
        sum_accuracy = float(acc.data) * test_data_num
        
        return float(loss.data), float(acc.data)
        
    train_mean_loss_list = []
    train_mean_acc_list = []
    test_mean_loss_list = []
    test_mean_acc_list = []
    


In [49]:
def train_and_eval(batchsize, n_epoch, print_epoch, drop_out, drop_out_ratio, intermediate_layer_num):  
    train_mean_loss_list = []
    train_mean_acc_list = []
    test_mean_loss_list = []
    test_mean_acc_list = []

    tmp_model = MultiLayerPerceptron(4, intermediate_layer_num, 3, drop_out_ratio=drop_out_ratio)
    mlp_model = L.Classifier(tmp_model)

    optimizer = optimizers.Adam()
    optimizer.setup(mlp_model)

    mb_learner = MiniBatchLearner(optimizer=optimizer, epoch_num=n_epoch, batch_size=batchsize)
    for epoch in range(1, n_epoch+1):
        if epoch % print_epoch == 0:
            print('epoch', epoch)

        # training
        train_mean_loss, train_mean_acc = mb_learner.learn(mlp_model, predictor_train, target_train, drop_out)
        if epoch % print_epoch == 0:
            print('train mean loss={}, accuracy={}'.format(train_mean_loss, train_mean_acc))

        # evaluation
        test_mean_loss, test_mean_acc = mb_learner.evaluate(mlp_model, predictor_test, target_test)    
        if epoch % print_epoch == 0:
            print('test mean loss={}, accuracy={}'.format(test_mean_loss, test_mean_acc))

        train_mean_loss_list.append(train_mean_loss)
        train_mean_acc_list.append(train_mean_acc)
        test_mean_loss_list.append(test_mean_loss)
        test_mean_acc_list.append(test_mean_acc)
    return train_mean_loss_list, train_mean_acc_list, test_mean_loss_list, test_mean_acc_list
def draw_loss_and_acc(train_mean_loss_list, train_mean_acc_list, test_mean_loss_list, test_mean_acc_list, xlabel="epoch", set_ylim=True):
    fig = plt.figure(figsize=(14, 5))

    # train
    ax1 = fig.add_subplot(1,2,1)
    plt.plot(range(len(train_mean_loss_list)), train_mean_loss_list)
    plt.plot(range(len(train_mean_acc_list)), train_mean_acc_list)
    plt.legend(["train_loss","train_acc"],loc=1)
    plt.title("Loss / Accuracy of Iris class recognition.")
    plt.xlabel(xlabel)
    plt.ylabel("loss/accracy")
    if set_ylim:
        ax1.set_ylim([0, 1.2])
    plt.grid()

    # test
    ax2 = fig.add_subplot(1,2,2)
    plt.plot(range(len(test_mean_loss_list)), test_mean_loss_list)
    plt.plot(range(len(test_mean_acc_list)), test_mean_acc_list)
    plt.legend(["test_loss","test_acc"],loc=1)
    plt.title("Loss / Accuracy of Iris class recognition.")
    plt.xlabel(xlabel)
    plt.ylabel("loss/accracy")
    if set_ylim:
        ax2.set_ylim([0, 1.2])
    plt.grid()

    plt.show()

In [50]:
batchsize = 10
n_epoch = 2000
print_epoch = n_epoch / 2
drop_out_ratio = 0.3
intermediate_layer_num=8
drop_out = True
train_mean_loss_list, train_mean_acc_list, test_mean_loss_list, test_mean_acc_list  = train_and_eval(batchsize, n_epoch, print_epoch, drop_out, drop_out_ratio, intermediate_layer_num)
draw_loss_and_acc(train_mean_loss_list, train_mean_acc_list, test_mean_loss_list, test_mean_acc_list)

ValueError: train argument is not supported anymore. Use chainer.using_config